Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [3]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [4]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [7]:
import os
import glob
from pathlib import Path

def gather_data(path_to_data): 
    data = []
    for f in glob.glob(str(Path(path_to_data)/'*.txt')):
        with open(f) as t:
            text = t.read().strip('\n')
            data.append(tokenize(str(text)))       
    return data

In [8]:
tokens = gather_data(path)

In [9]:
tokens[0][0:10]

['ear',
 'willingly',
 'small',
 'possessed',
 'spell',
 'bound',
 'attention',
 'preached',
 'romanism',
 'persuaded']

In [10]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

## Follow Along

In [11]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [12]:
titles

['CBronte_Villette0145',
 'Austen_Sense0002',
 'Austen_Sense0016',
 'CBronte_Villette0151',
 'Austen_Emma0128',
 'Austen_Emma0100',
 'Austen_Emma0114',
 'CBronte_Jane0089',
 'CBronte_Professor0026',
 'CBronte_Jane0062',
 'CBronte_Jane0076',
 'CBronte_Professor0032',
 'Austen_Emma0048',
 'Austen_Emma0074',
 'Austen_Emma0060',
 'CBronte_Jane0116',
 'CBronte_Jane0102',
 'CBronte_Villette0031',
 'CBronte_Villette0025',
 'CBronte_Villette0019',
 'Austen_Pride0016',
 'Austen_Pride0002',
 'Austen_Pride0003',
 'Austen_Pride0017',
 'CBronte_Villette0018',
 'CBronte_Villette0024',
 'CBronte_Villette0030',
 'CBronte_Jane0103',
 'CBronte_Jane0117',
 'Austen_Emma0061',
 'Austen_Emma0075',
 'Austen_Emma0049',
 'CBronte_Jane0077',
 'CBronte_Professor0033',
 'CBronte_Professor0027',
 'CBronte_Jane0063',
 'CBronte_Jane0088',
 'Austen_Emma0115',
 'Austen_Emma0101',
 'Austen_Emma0129',
 'Austen_Sense0017',
 'CBronte_Villette0150',
 'CBronte_Villette0144',
 'Austen_Sense0003',
 'CBronte_Villette0152',
 'A

In [13]:
len(titles)

813

In [14]:
len(tokens)

813

### Author DataFrame


In [15]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [16]:
df.head()

,tokens
CBronte_Villette0145,"[ear, willingly, small, possessed, spell, boun..."
Austen_Sense0002,"[half, blood, generous, spirit, wish, thing, m..."
Austen_Sense0016,"[determine, intimacy, disposition, seven, year..."
CBronte_Villette0151,"[louisa, carriage, stature, fit, princess, com..."
Austen_Emma0128,"[delightful, assurance, total, indifference, f..."


In [17]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [18]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [19]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [20]:
df.head()

,tokens,author,book,section
CBronte_Villette0145,"[ear, willingly, small, possessed, spell, boun...",0,Villette,145
Austen_Sense0002,"[half, blood, generous, spirit, wish, thing, m...",1,Sense,2
Austen_Sense0016,"[determine, intimacy, disposition, seven, year...",1,Sense,16
CBronte_Villette0151,"[louisa, carriage, stature, fit, princess, com...",0,Villette,151
Austen_Emma0128,"[delightful, assurance, total, indifference, f...",1,Emma,128


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [21]:
def doc_stream(path):
    data = []
    for f in glob.glob(str(Path(path)/'*.txt')):
        with open(f) as t:
            text = t.read().strip('\n')
            tokens = tokenize(text)
        yield tokens

In [22]:
streaming_data = doc_stream(path)

In [23]:
type(streaming_data)

generator

In [24]:
# gather_data => returns a list
# doc_stream => returns a generator

In [25]:
next(streaming_data) # Returns one document at a time from the generator

['ear',
 'willingly',
 'small',
 'possessed',
 'spell',
 'bound',
 'attention',
 'preached',
 'romanism',
 'persuaded',
 'conversion',
 'voice',
 'sly',
 'little',
 'book',
 'honeyed',
 'voice',
 'accents',
 'unction',
 'balm',
 'roared',
 'utterance',
 'rome',
 'thunders',
 'blasting',
 'breath',
 'displeasure',
 'protestant',
 'turn',
 'papist',
 'fear',
 'heretic',
 'hell',
 'account',
 'comfort',
 'indulgence',
 'tenderness',
 'holy',
 'church',
 'offered',
 'far',
 'threaten',
 'coerce',
 'wish',
 'guide',
 'win',
 'persecute',
 'oh',
 'dear',
 'account',
 'meek',
 'volume',
 'addressed',
 'hardened',
 'worldly',
 'strong',
 'meat',
 'strong',
 'milk',
 'babes',
 'mild',
 'effluence',
 'mother',
 'love',
 'tenderest',
 'youngest',
 'intended',
 'wholly',
 'solely',
 'head',
 'reached',
 'heart',
 'appeal',
 'intellect',
 'sought',
 'win',
 'affectionate',
 'affections',
 'sympathizing',
 'sympathies',
 'st',
 'vincent',
 'paul',
 'gathering',
 'orphans',
 'spoke',
 'sweetly',
 're

### Gensim LDA Topic Modeling

In [26]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [27]:
id2word.token2id['england']

2172

In [28]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(2072, 1), (2172, 3), (3587, 1), (15868, 1)]

In [29]:
id2word[6818]

'repress'

In [30]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
7056


In [31]:
len(id2word.keys())

22095

In [32]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [33]:
len(id2word.keys())

8102

In [34]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [35]:
corpus[345][:10]

[(46, 1),
 (68, 1),
 (69, 1),
 (98, 1),
 (104, 1),
 (116, 1),
 (137, 1),
 (152, 2),
 (154, 5),
 (160, 1)]

In [36]:
id2word[88]

'east'

In [37]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )
# LDA latent dirchilect allocation
# Dirchilect In probability and statistics, the Dirichlet distribution 
# (after Peter Gustav Lejeune Dirichlet), often denoted , is a family of 
# continuous multivariate probability distributions parameterized by a vector. 
# of positive reals.

In [38]:
lda.print_topics()
#every element is a string with a topic number (percentage of each word and contains only 10 most common words)

[(0,
  '0.007*"like" + 0.005*"know" + 0.005*"thought" + 0.004*"think" + 0.004*"good" + 0.004*"rochester" + 0.004*"face" + 0.003*"eye" + 0.003*"house" + 0.003*"little"'),
 (1,
  '0.001*"miss" + 0.001*"little" + 0.000*"think" + 0.000*"know" + 0.000*"like" + 0.000*"shall" + 0.000*"day" + 0.000*"time" + 0.000*"emma" + 0.000*"better"'),
 (2,
  '0.011*"miss" + 0.010*"emma" + 0.009*"elton" + 0.008*"think" + 0.008*"weston" + 0.008*"smith" + 0.008*"harriet" + 0.007*"good" + 0.006*"thought" + 0.005*"thing"'),
 (3,
  '0.015*"graham" + 0.013*"bretton" + 0.011*"papa" + 0.009*"lucy" + 0.008*"little" + 0.007*"like" + 0.006*"polly" + 0.005*"paulina" + 0.005*"know" + 0.005*"bassompierre"'),
 (4,
  '0.007*"little" + 0.005*"monsieur" + 0.005*"madame" + 0.005*"time" + 0.004*"door" + 0.004*"like" + 0.004*"shall" + 0.004*"hand" + 0.004*"day" + 0.004*"looked"'),
 (5,
  '0.005*"like" + 0.004*"day" + 0.004*"little" + 0.004*"time" + 0.003*"half" + 0.003*"saw" + 0.003*"long" + 0.003*"reed" + 0.003*"thought" + 0.

In [40]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
# removes numbers and spaces and * 

In [41]:
topics = [' '.join(t[0:5]) for t in words]
# then joins the remaining words line by line

In [42]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")
    
# can use named entity from spacy and this can identify names of people. 

------ Topic 0 ------
like know thought think good

------ Topic 1 ------
miss little think know like

------ Topic 2 ------
miss emma elton think weston

------ Topic 3 ------
graham bretton papa lucy little

------ Topic 4 ------
little monsieur madame time door

------ Topic 5 ------
like day little time half

------ Topic 6 ------
jane sir like shall night

------ Topic 7 ------
rochester room little miss come

------ Topic 8 ------
little lydia like uncle wickham

------ Topic 9 ------
elinor marianne miss know time

------ Topic 10 ------
like little thought know good

------ Topic 11 ------
little miss day like time

------ Topic 12 ------
elizabeth place thought pemberley fine

------ Topic 13 ------
elizabeth darcy bennet bingley jane

------ Topic 14 ------
emma harriet miss knightley elton



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [43]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

#LDA visualization

In [44]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

#red word - word in topic
#blue word - word in corpus

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.033568  0.010115       1        1  21.760263
9      0.127375  0.010082       2        1  20.987228
14     0.165061  0.032316       3        1  11.727550
13     0.152205  0.018256       4        1  10.921200
6     -0.057278  0.062462       5        1   7.628895
4     -0.069070  0.053162       6        1   7.058738
0     -0.022956  0.029391       7        1   5.431089
7     -0.033165  0.064219       8        1   4.387336
11    -0.086523  0.037231       9        1   4.381789
3     -0.039921  0.068848      10        1   3.074739
5     -0.098336  0.005568      11        1   1.228189
8     -0.010526 -0.037171      12        1   0.599207
2      0.110594 -0.070176      13        1   0.529118
12    -0.039769 -0.137420      14        1   0.267394
1     -0.064123 -0.146884      15        1   0.017262, topic_info=           Term         Freq        Total Category  logprob  loglift
1021       emma   911.000000   911.000000  Default  30.0000  30.0000
2976  elizabeth   652.000000   652.000000  Default  29.0000  29.0000
1047    harriet   559.000000   559.000000  Default  28.0000  28.0000
678        miss  1547.000000  1547.000000  Default  27.0000  27.0000
163      little  1717.000000  1717.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
434       great     0.015735   825.283875  Topic15  -8.1428  -2.2032
304     thought     0.016462  1137.389160  Topic15  -8.0977  -2.4788
198          oh     0.014597   566.766479  Topic15  -8.2178  -1.9024
432        good     0.016033  1359.289795  Topic15  -8.1241  -2.6834
820        eyes     0.014699   635.013245  Topic15  -8.2109  -2.0092

[1134 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4686      3  0.960458    abbey
6045      1  0.320457  ability
6045      2  0.480686  ability
6045     14  0.160229  ability
5792      1  0.191273   abrupt
...     ...       ...      ...
542       8  0.029891    young
542       9  0.029891    young
542      10  0.014945    young
542      12  0.001495    young
542      13  0.010462    young

[4233 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 10, 15, 14, 7, 5, 1, 8, 12, 4, 6, 9, 3, 13, 2])

### Overall Model / Documents

<img src =https://slideplayer.com/slide/4559649/15/images/3/Statistical+topic+modeling+%282%2F3%29.jpg>

[0.25, 0.15, 0.45, 0.15]
LDA belief is that with this distribution you can recreate the corpus (with the BOW assumption - order is not preserved)

**Alpha** controls the number of topics retained in a corpus 

**Beta** small number of words to have a high probability

Rule of thumb have below 1 but not too close to 0

In [59]:
lda[corpus[0]] 
#- proportion of topics 

[(6, 0.10002042), (10, 0.7503668), (11, 0.1476825)]

In [46]:
distro = [lda[d] for d in corpus]

In [47]:
distro[0]

[(6, 0.100028604), (10, 0.75035405), (11, 0.14768709)]

In [48]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [49]:
len(new_distro)

813

In [50]:
df.head()

,tokens,author,book,section
CBronte_Villette0145,"[ear, willingly, small, possessed, spell, boun...",0,Villette,145
Austen_Sense0002,"[half, blood, generous, spirit, wish, thing, m...",1,Sense,2
Austen_Sense0016,"[determine, intimacy, disposition, seven, year...",1,Sense,16
CBronte_Villette0151,"[louisa, carriage, stature, fit, princess, com...",0,Villette,151
Austen_Emma0128,"[delightful, assurance, total, indifference, f...",1,Emma,128


In [51]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [52]:
df.head()

,like know thought think good,miss little think know like,miss emma elton think weston,graham bretton papa lucy little,little monsieur madame time door,like day little time half,jane sir like shall night,rochester room little miss come,little lydia like uncle wickham,elinor marianne miss know time,like little thought know good,little miss day like time,elizabeth place thought pemberley fine,elizabeth darcy bennet bingley jane,emma harriet miss knightley elton,author
CBronte_Villette0145,0.0,0,0.0,0.000000,0.0,0.0,0.100025,0.0,0.0,0.000000,0.750362,0.147682,0.0,0.0,0.000000,CBronte
Austen_Sense0002,0.0,0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.997054,0.000000,0.000000,0.0,0.0,0.000000,Austen
Austen_Sense0016,0.0,0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.997587,0.000000,0.000000,0.0,0.0,0.000000,Austen
CBronte_Villette0151,0.0,0,0.0,0.337546,0.0,0.0,0.368355,0.0,0.0,0.000000,0.292224,0.000000,0.0,0.0,0.000000,CBronte
Austen_Emma0128,0.0,0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.470927,0.011310,0.000000,0.0,0.0,0.515786,Austen


In [53]:
df.groupby('author').mean()

,like know thought think good,miss little think know like,miss emma elton think weston,graham bretton papa lucy little,little monsieur madame time door,like day little time half,jane sir like shall night,rochester room little miss come,little lydia like uncle wickham,elinor marianne miss know time,like little thought know good,little miss day like time,elizabeth place thought pemberley fine,elizabeth darcy bennet bingley jane,emma harriet miss knightley elton
author,,,,,,,,,,,,,,,
Austen,0.007903,0,0.011373,0.000211,0.000080,0.000000,0.002262,0.004038,0.003599,0.443846,0.007421,0.001275,0.002682,0.257707,0.254971
CBronte,0.094770,0,0.000000,0.053266,0.124877,0.021019,0.141331,0.076424,0.008168,0.017054,0.378825,0.075036,0.002262,0.001259,0.003025


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [54]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

# Idea behind coherence value is higher when there are a few dominate topics 

In [ ]:
# Can take a long time to run.
%time
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=100)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 23.1 µs


In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)